You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull

!cd k && pip install .

Already up to date.
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=56706 sha256=b3c3e15961b3710ff71f610a2e5f2da25a8b01dac7245213450ec9545d7ed1f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-42_va7dx/wheels/c1/8a/57/56dbba25eff58e52e5365435c4fa102ad8d6f9787d3b4db13a
Successfully built cai
  Attempting uninstall: cai
    Found existing installation: cai 0.1.4
    Uninstalling cai-0.1.4:
      Successfully uninstalled cai-0.1.4


In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.6.0
Keras version: 2.6.0


In [ ]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [ ]:
#mixed_precision.set_global_policy('mixed_float16')

In [ ]:
dataset=tf.keras.datasets.cifar10
verbose=True
lab=False
bipolar=False
base_model_name='best'
x_train, y_train, x_test, y_test = cai.datasets.load_dataset(dataset, verbose=verbose, lab=lab, bipolar=bipolar, base_model_name=base_model_name)
print(x_train.shape)
print(y_train.shape)

train shape (50000, 32, 32, 3)
test shape (10000, 32, 32, 3)
Original channel  0  min: 0.0  max: 255.0
Original channel  1  min: 0.0  max: 255.0
Original channel  2  min: 0.0  max: 255.0
Loading RGB.
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0
(50000, 32, 32, 3)
(50000, 10)


In [ ]:
num_classes = 10
batch_size = 64
epochs = 50
target_size_x = 32
target_size_y = 32
seed = 12
input_shape = (32, 32, 3)

In [ ]:
train_datagen = cai.util.create_image_generator(validation_split=0.1, rotation_range=20, width_shift_range=0.3, height_shift_range=0.3)
test_datagen = cai.util.create_image_generator_no_augmentation()
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [ ]:
batches_per_epoch = int(45000/batch_size)
l2_decay = 0.0001 / batches_per_epoch

In [ ]:
import tensorflow
import tensorflow.keras

In [ ]:
def work_on_densenet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [0, 23, 25]:
      basefilename = 'JP33B15-kDenseNet-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == 0:
        model = cai.densenet.simple_densenet([32, 32, 3], blocks=16, 
          growth_rate=12, bottleneck=48, compression=0.5,
          l2_decay=0, num_classes=num_classes, dropout_rate=0.0)
      else:
        model = cai.densenet.ksimple_densenet([32, 32, 3], blocks=16, 
          growth_rate=12, bottleneck=48, compression=0.5,
          l2_decay=0, kTypeBlock=kType, kTypeTransition=kType, 
          num_classes=num_classes, dropout_rate=0.0,
          activation=keras.activations.relu)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): 
        model.summary(line_length=220)
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow(
                x_train, y_train,
                batch_size=batch_size,
                shuffle=True,
                seed=seed,
                subset='training'
            )
        validation_flow = train_datagen.flow(
                x_train, y_train,
                batch_size=batch_size,
                shuffle=True,
                seed=seed,
                subset='validation'
            )

        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow(
            x_test, y_test,
            batch_size=batch_size,
            shuffle=True,
            seed=seed
            )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

In [ ]:
work_on_densenet(show_model=True, run_fit=False, test_results=False)

Running: JP33B15-kDenseNet-CIFAR10-0
Model: "model"
____________________________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                            Output Shape                                     Param #                   Connected to                                                             
input_1 (InputLayer)                                                    [(None, 32, 32, 3)]                              0                                                                                                  
____________________________________________________________________________________________________________________________________________________________________________________________________________________________
conv2d (Conv2D)                                                 

In [ ]:
work_on_densenet(show_model=False, run_fit=True, test_results=False)

Running: JP33B15-kDenseNet-CIFAR10-0
Epoch 1/50
704/704 [==============================] - 132s 154ms/step - loss: 1.5639 - accuracy: 0.4323 - val_loss: 1.7433 - val_accuracy: 0.4302

Epoch 00001: val_accuracy improved from -inf to 0.43020, saving model to JP33B15-kDenseNet-CIFAR10-0-best_result.hdf5
Epoch 2/50
704/704 [==============================] - 106s 150ms/step - loss: 1.2584 - accuracy: 0.5529 - val_loss: 2.0087 - val_accuracy: 0.4018

Epoch 00002: val_accuracy did not improve from 0.43020
Epoch 3/50
704/704 [==============================] - 106s 149ms/step - loss: 1.1107 - accuracy: 0.6093 - val_loss: 1.4308 - val_accuracy: 0.5278

Epoch 00003: val_accuracy improved from 0.43020 to 0.52780, saving model to JP33B15-kDenseNet-CIFAR10-0-best_result.hdf5
Epoch 4/50
704/704 [==============================] - 105s 149ms/step - loss: 0.9928 - accuracy: 0.6504 - val_loss: 1.4360 - val_accuracy: 0.5418

Epoch 00004: val_accuracy improved from 0.52780 to 0.54180, saving model to JP33B

In [ ]:
work_on_densenet(show_model=False, run_fit=False, test_results=True)

Running: JP33B15-kDenseNet-CIFAR10-0
Best Model Results: JP33B15-kDenseNet-CIFAR10-0-best_result.hdf5
157/157 [==============================] - 6s 31ms/step - loss: 0.2828 - accuracy: 0.9238
loss 0.2827649414539337
acc 0.923799991607666
Finished: JP33B15-kDenseNet-CIFAR10-0
Running: JP33B15-kDenseNet-CIFAR10-23
Best Model Results: JP33B15-kDenseNet-CIFAR10-23-best_result.hdf5
157/157 [==============================] - 7s 34ms/step - loss: 0.3257 - accuracy: 0.9041
loss 0.3257104754447937
acc 0.9041000008583069
Finished: JP33B15-kDenseNet-CIFAR10-23
Running: JP33B15-kDenseNet-CIFAR10-25
Best Model Results: JP33B15-kDenseNet-CIFAR10-25-best_result.hdf5
157/157 [==============================] - 6s 32ms/step - loss: 0.2762 - accuracy: 0.9197
loss 0.27621224522590637
acc 0.919700026512146
Finished: JP33B15-kDenseNet-CIFAR10-25
